In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
browser = webdriver.Chrome('windows/chromedriver')

In [3]:
url = 'https://www.glassdoor.com/Job/washington-data-scientist-jobs-SRCH_IL.0,4_IS2235_KO5,19.htm?radius=25&fromAge=30'
browser.get(url)


In [4]:
# def geturls():
#     url_set = set()
#     base_url = 'https://www.glassdoor.com'
#     pages = soup.find_all('li', class_='page')

#     for page in pages:
#         if page.a:
#             url = base_url + page.a['href']
#             if (url not in url_set) :
#                  url_set.add(url)
               
#     return url_set

In [5]:
def trigger_and_close_popup(browser):
    
    xpath= '//*[@id="MainCol"]/div/ul/li[1]'
    browser.find_element_by_xpath(xpath).click()
    time.sleep(1)

    xpath ='//*[@id="JAModal"]/div/div/div[2]/div/div[1]/div/span'
    browser.find_element_by_xpath(xpath).click()
    time.sleep(1)


In [6]:
def soup_for_click(xpath):
    browser.find_element_by_xpath(xpath).click()
    time.sleep(1)
    html = browser.page_source
    return BeautifulSoup(html, 'html.parser')

In [ ]:
continue_bool = True
jobs = []

while True:

    #trigger popup and close it
    try:
        trigger_and_close_popup(browser)
    except Exception as e:
        pass
    
#     close any possible popups with a Close button
    try:
        browser.find_element_by_partial_link_text('Close').click()
        time.sleep(1)
    except Exception as e:
        pass
    
#     more popups
    try:
        browser.find_element_by_class_name('ModalStyle__xBtn___29PT9').click()
        time.sleep(1)
    except Exception as e:
        pass
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    
    article = soup.find('article', id='MainCol').ul
    job_list = article.find_all('li')
 
    for job  in job_list:
    #     title
        title = job['data-normalize-job-title'].strip()
    #     job location
        location  = job['data-job-loc'].strip()
    #     employer
        employer = job.find('div', class_='jobEmpolyerName').text
      
    #     job details
        dataId = job['data-id']
        Id = 'JobDesc' + dataId
        xpath = f"//li[@data-id='{dataId}']"
        try:
            soup = soup_for_click(xpath)
            job_desc = soup.find('div', id = Id, class_="jobDesc").text
        except Exception as e:
            job_desc = "" 

    #     review count
        xpath= '//*[@id="Details"]/div[1]/div[1]/div/div/div[5]'
        try:
            soup = soup_for_click(xpath)
            reviews= soup.select("li.empReview")
            review_count = len(reviews)
        except Exception as e:
            review_count = 0

    #   create the job dictoanry object and add it to the list
        job_dict = {"position": title,"company": employer, "description": job_desc, "reviews": review_count, \
                    "location":location}

        jobs.append(job_dict)
        
    try:
        xpath = '//*[@id="FooterPageNav"]/div/ul/li[7]/a'
        browser.find_element_by_xpath(xpath).click()
        time.sleep(1)
        
    except NoSuchElementException:
        break

print("Scraping Complete")
      
        
        



In [ ]:
#export to excel without the df index
glassdoor_df = pd.DataFrame(jobs)
glassdoor_df.to_excel("C:/Users/indup/ETL-Project/Transform/Resources/Glassdoor/wa_glassdoor.xlsx", index=False)
 
